In [1]:
from bs4 import BeautifulSoup
import requests
from IPython.core.display import display, HTML
import re
import dateutil.parser

In [2]:
url = 'https://www.boxofficemojo.com/title/tt0118715/'
response = requests.get(url)
page = response.text
soup = BeautifulSoup(page, "lxml")

title_string = soup.find('title').text
title_string.split('-')
title = title_string.split('-')[0].strip()
dtg = soup.find(class_='mojo-performance-summary-table').find_all('span', class_='money')[0].text



In [3]:
def get_movie_value(soup, field_name):
    '''Grab a value from Box Office Mojo HTML
    
    Takes a string attribute of a movie on the page and returns the string in
    the next sibling object (the value for that attribute) or None if nothing is found.
    '''
    obj = soup.find(text=re.compile(field_name))
    if not obj: 
        return None
    # this works for most of the values
    next_element = obj.findNext()
    if next_element:
        return next_element.text 
    else:
        return None

In [4]:
runtime = get_movie_value(soup,'Run')
rating = get_movie_value(soup,'MPAA')
release_date = get_movie_value(soup,'Release Date')
release_date = release_date.split('\n')[0]

In [5]:
import dateutil.parser

def money_to_int(moneystring):
    moneystring = moneystring.replace('$', '').replace(',', '')
    return int(moneystring)

def runtime_to_minutes(runtimestring):
    runtime = runtimestring.split()
    try:
        minutes = int(runtime[0])*60 + int(runtime[2])
        return minutes
    except:
        return None

def to_date(datestring):
    date = dateutil.parser.parse(datestring)
    return date

In [13]:
raw_domestic_total_gross = dtg
domestic_total_gross = money_to_int(raw_domestic_total_gross)

raw_runtime = get_movie_value(soup,'Running')
runtime = runtime_to_minutes(raw_runtime)

raw_release_date = get_movie_value(soup,'Release Date').split('\n')[0]
release_date = to_date(raw_release_date)

In [7]:
headers = ['movie title', 'domestic total gross',
           'runtime (mins)', 'rating', 'release date']

movie_data = []
movie_dict = dict(zip(headers, [title,
                                domestic_total_gross,
                                runtime,
                                rating, 
                                release_date]))

movie_data.append(movie_dict)
movie_data

[{'movie title': 'The Big Lebowski',
  'domestic total gross': 18034458,
  'runtime (mins)': 117,
  'rating': 'R',
  'release date': datetime.datetime(1998, 3, 6, 0, 0)}]

In [9]:
type(movie_data[0])

dict